In [1]:
%load_ext autoreload

%autoreload

import numpy as np 
import pandas as pd 


data_1 = pd.read_csv('data_1.csv')
data_1

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [29]:
import numpy as np
from math import log

# Define calculate_entropy function to make things easier
def entropy(counts):
    """
    Computes the entropy of a partitioning
    
    Args:
        counts (array<k>): a lenth k int array >= 0. For instance,
            an array [3, 4, 1] implies that you have a total of 8
            datapoints where 3 are in the first group, 4 in the second,
            and 1 one in the last. This will result in entropy > 0.
            In contrast, a perfect partitioning like [8, 0, 0] will
            result in a (minimal) entropy of 0.0
            
    Returns:
        A positive float scalar corresponding to the (log2) entropy
        of the partitioning.
    
    """
    assert (counts >= 0).all()
    probs = counts / counts.sum()
    probs = probs[probs > 0]  # Avoid log(0)
    return - np.sum(probs * np.log2(probs))



 

def split_by_feature(X, feature_idx, threshold):
    # if the feature is numerical
    if isinstance(threshold, int) or isinstance(threshold, float):
        X_true = X[X.iloc[:,feature_idx] >= threshold]
        X_false = X[~X.iloc[:,feature_idx] >= threshold]
    # if the feature is categorical
    else:
        X_true = X[X.iloc[:,feature_idx] == threshold]
        X_false = X[~(X.iloc[:,feature_idx] == threshold)]
        
    return X_true, X_false



class DecisionNode():
    def __init__(self, feature_idx=None, threshold=None, value=None, true_branch=None, false_branch=None):
        self.feature_idx = feature_idx # index of the feature that is used
        self.threshold = threshold # threshold value for feature when making the decision
        self.value = value # value if the node is a leaf in the tree
        self.true_branch = true_branch # the node we go to if decision returns True
        self.false_branch = false_branch # the node we go to if decision returns False
        
class DecisionTree():
    def __init__(self, min_info_gain=1e-7, max_depth=float("inf")):
        self.root = None # root of this tree
        self.min_info_gain = min_info_gain # minimum information gain to allow splitting
        self.max_depth = max_depth # maximum depth the tree grows to
    def fit(self, X, y):
        self.root = self.build_tree(X, y)
    def build_tree(self, X, y, current_depth=0):
        decision = None
        subtrees = None
        largest_info_gain = 0
        # add y as last column of X
        df = pd.concat((X, y), axis=1)
        n_rows, n_features = X.shape
        if current_depth <= self.max_depth:
            # iterate through every feature
            for feature_idx in range(n_features):
                # values of that column
                feature_values = X.iloc[:, feature_idx]
                unique_values = feature_values.unique()
                for threshold in unique_values:
                    X_true, X_false = split_by_feature(df, feature_idx, threshold)
                    if len(X_true) > 0 and len(X_false) > 0:
                        y_true = X_true.iloc[:,-1]
                        y_false = X_false.iloc[:,-1]
                        # Calculate impurity
                        info_gain = self.calculate_information_gain(y, y_true, y_false)
                        # Keep track of which feature gave the largest information gain
                        if info_gain > largest_info_gain:
                            largest_info_gain = info_gain
                            decision = {"feature_idx":feature_idx, "threshold":threshold}
                            subtrees = {"X_true":X_true.iloc[:,:-1],
                                        "y_true":y_true,
                                        "X_false":X_false.iloc[:,:-1],
                                        "y_false":y_false}
        # we will construct new branch if the information gain is larger than minimum information gain that we've defined
        if largest_info_gain > self.min_info_gain:
            true_branch = self.build_tree(subtrees["X_true"], subtrees["y_true"], current_depth+1)
            false_branch = self.build_tree(subtrees["X_false"], subtrees["y_false"], current_depth+1)
            return DecisionNode(feature_idx=decision["feature_idx"], threshold=decision["threshold"], true_branch=true_branch, false_branch=false_branch)

        # at leaf
        leaf_value = self.majority_vote(y)
        return DecisionNode(value=leaf_value)
                        
    def calculate_information_gain(self, y, y_true, y_false):
        # probability of choosing left subtree 
        p = len(y_true) / len(y)
        entropy = calculate_entropy(y)
        info_gain = entropy - p*calculate_entropy(y_true) - (1-p)*calculate_entropy(y_false)
        return info_gain
                
    def majority_vote(self, y):
        # this is for calculating values for the leaf nodes
        return y.value_counts().idxmax()
                
    def predict_value(self, x, tree=None):
        # recursive method to find the leaf node that corresponds to prediction
        if tree is None:
            tree = self.root
        if tree.value is not None:
            return tree.value
        feature_value = x[tree.feature_idx]
        branch = tree.false_branch
        if isinstance(feature_value, int) or isinstance(feature_value, float):
            if feature_value >= tree.threshold:
                branch = tree.true_branch
        elif feature_value == tree.threshold:
            branch = tree.true_branch
        return self.predict_value(x, branch)
    
    def predict(self, X):
        y_pred = []
        for idx, row in X.iterrows():
            y_pred.append(self.predict_value(row.values))
        return y_pred



def accuracy(y_true, y_pred):
    """
    Computes discrete classification accuracy
    
    Args:
        y_true (array<m>): a length m vector of ground truth labels
        y_pred (array<m>): a length m vector of predicted labels
        
    Returns:
        The average number of correct predictions
    """
    assert y_true.shape == y_pred.shape
    return (y_true == y_pred).mean()

In [30]:
# Separate independent (X) and dependent (y) variables
X = data_1.drop(columns=['Play Tennis'])
y = data_1['Play Tennis']

# Create and fit a Decrision Tree classifier
model_1 = DecisionTree()  # <-- Should work with default constructor
model_1.fit(X, y)

# Verify that it perfectly fits the training set
print(f'Accuracy: {accuracy(y_true=np.array(y), y_pred=np.array(model_1.predict(X))) * 100 :.1f}%')

Accuracy: 100.0%


In [ ]:
#My attempt at an implementation:
class TreeNode():
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None): #asterix to force caller to explictily write value="something"
        self.feature   = feature
        self.threshold = threshold
        self.left      = left
        self.right     = right
        self.value     = value
        
    def isLeafNode(self):
        return self.value is not None
    
class DecisionTree():
    def __init__(self, max_tree_depth=np.inf):
        self.root = None
        self.max_tree_depth = max_tree_depth
        
    def fit(self, X, y):
        """
        Generates a decision tree for classification
        
        Args:
            X (pd.DataFrame): a matrix with discrete value where
                each row is a sample and the columns correspond
                to the features.
            y (pd.Series): a vector of discrete ground-truth labels
        """
        #start at the top node and at each node select the best split based on the best
        #information gain
        
        #Greedy search: Loop over all features and over all thresholds and
        #save the best split feature and split threshold at each node
        
        #Build the tree recursively
        
        #apply some stopping criteria to stop growing:
        #For example, maximum depth, minimum samples at node, no more class
        #distribution in node
        
        #when we have a leaf node store the most frequent class label of this node
        
        
        
        # TODO: Implement 
        raise NotImplementedError()
    
    def predict(self, X):
        """
        Generates predictions
        
        Note: should be called after .fit()
        
        Args:
            X (pd.DataFrame): an mxn discrete matrix where
                each row is a sample and the columns correspond
                to the features.
            
        Returns:
            A length m vector with predictions
        """
        
        #Traverse the tree recursively
        #At each node look at the best split feature of the test feature vector x
        #and go left or right depending on x[feature_idx] <= threshold
        
        #when we reach the leaf node we return the stored most common class label
        
        
        # TODO: Implement 
        raise NotImplementedError()
        
    
    
    

    
    
    
def entropy(counts):
    """
    Computes the entropy of a partitioning
    
    Args:
        counts (array<k>): a lenth k int array >= 0. For instance,
            an array [3, 4, 1] implies that you have a total of 8
            datapoints where 3 are in the first group, 4 in the second,
            and 1 one in the last. This will result in entropy > 0.
            In contrast, a perfect partitioning like [8, 0, 0] will
            result in a (minimal) entropy of 0.0
            
    Returns:
        A positive float scalar corresponding to the (log2) entropy
        of the partitioning.
    
    """
    assert (counts >= 0).all()
    probs = counts / counts.sum()
    probs = probs[probs > 0]  # Avoid log(0)
    return - np.sum(probs * np.log2(probs))


